<div class="alert alert-success">

#### Homework 8 Supplemental Notebook
    
# Multiple Linear Regression

### EECS 245, Fall 2025 at the University of Michigan
    
</div>

### Instructions

Most homeworks will have Jupyter Notebooks, like this one, designed to supplement the theoretical problems. 

To write and run code in this notebook, you have two options:

1. **Use the EECS 245 DataHub.** To do this, click the link provided in the Homework 8 PDF. Before doing so, read the instructions on the [**Tech Support**](https://eecs245.org/tech-support/#option-1-using-the-eecs-245-datahub) page on how to use the DataHub.
1. **Set up a Jupyter Notebook environment locally, and use `git` to clone our course repository.** For instructions on how to do this, see the [**Tech Support**](https://eecs245.org/tech-support) page of the course website.

**You do not need to submit this notebook to Gradescope.** Instead, you'll be told to screenshot your implementations of various tasks and include them in your Homework 8 PDF.

In [ ]:
# Run this cell.
import numpy as np
import pandas as pd
import time

pd.options.plotting.backend = "plotly"

import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Set default layout for all plotly figures
import plotly.graph_objs as go

custom_template = go.layout.Template(pio.templates["plotly_white"])
custom_template.layout.plot_bgcolor = "white"
custom_template.layout.paper_bgcolor = "white"
custom_template.layout.margin = dict(l=60, r=60, t=60, b=60)
custom_template.layout.width = 700
custom_template.layout.font = dict(
    family="Palatino Linotype, Palatino, serif",
    color="black"
)

pio.templates["custom"] = custom_template
pio.templates.default = "custom"

## Problem 4: The Complete Solution

---

We're intentionally not providing any starter code for this problem. Instead, peek through [Chapter 3.1](https://notes.eecs245.org/multiple-linear-regression/regression-using-linear-algebra/) and [Chapter 3.2](https://notes.eecs245.org/multiple-linear-regression/multiple-linear-regression/) of the course notes, or later in this notebook, to see how to write the relevant code. It should only take a few lines.

## Problem 5: Home Run!

---

Below, we load in a dataset that describes the [number of home runs in the MLB per year](https://www.mlb.com/glossary/standard-stats/home-run). 

In [ ]:
homeruns = pd.read_csv('data/homeruns.csv')
homeruns

In [ ]:
fig = homeruns.plot(kind='scatter', x='Year', y='Homeruns', title='Homeruns vs. Year')
fig.show(renderer='notebook')

Remember, this problem is **not autograded**. Instead, in each part, include screenshots of your code and a scatter plot of your model's predictions, and write the formula for your fit model, in your Homework 8 PDF.

### Problem 5a)

$$h(x_i) = w_0 + w_1 x_i + w_2 x_i^2$$

In [ ]:
from sklearn.linear_model import LinearRegression # Only needs to be imported once.
from sklearn.metrics import mean_squared_error

def plot_raw_data_and_predictions(X_raw, X_proc, y, model):
    """
    Inputs:
        - X_raw, a DataFrame with a single column, `Year`
        - X_proc, a DataFrame that results from transforming X_raw with a feature creator
        - y, a Series with `Homeruns` values
        - model, an **already fit** `LinearRegression` objects
    Outputs:
        - Returns a plotly figure object showing a scatter plot of the raw data in blue, with the model's predictions overlaid in orange

    """
    # Reuse me!
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=X_raw.to_numpy().flatten(),
            y=y,
            mode='markers',
            name='Actual'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=X_raw.to_numpy().flatten(),
            y=model.predict(X_proc),
            mode='lines',
            name='Predictions',
            line=dict(color='orange', width=4)
        )
    )
    fig.update_layout(
        title='Homeruns vs. Year (with Model Predictions)',
        xaxis_title='Year',
        yaxis_title='Homeruns'
    )
    return fig

def process_and_fit_model(X, y, feature_creator):
    """
    Inputs:
        - X, a DataFrame with a single column, `Year`
        - y, a Series with `Homeruns` values
        - feature_creator, a function that takes in X and returns a DataFrame with the new added features
    Outputs:
        - Fits a `LinearRegression` model on the transformed data, prints the optimal parameters, and prints the mean squared error of the model's predictions
        - Draws a plot of the raw data in blue, with the model's predictions overlaid in orange
    """

    # Create features
    X_raw = X.copy()
    X_proc = feature_creator(X)

    # Instantiate and fit model
    model = LinearRegression()
    model.fit(X=X_proc, y=y)

    # Compute predictions and mean squared error
    preds = model.predict(X=X_proc)
    rmse = mean_squared_error(y, preds) ** 0.5
    print('optimal parameters:', model.intercept_, model.coef_)
    print('root mean squared error:', rmse)
    print('r^2', model.score(X_proc, y))

    # Draw plot of predictions
    fig = plot_raw_data_and_predictions(X_raw, X_proc, y, model)
    fig.show(renderer='notebook')

def create_feature_columns_5a(X):
    """
    Inputs:
        - X, a DataFrame with a single column, `Year`
    Outputs:
        - Returns a DataFrame with two columns: `Year` and `Year^2`
    """       
    X = X.copy() # Don't forget this; otherwise, you may make in-place modifications to the raw data.
    X['Year'] = X['Year'] # Already there; just including this line to show that it hasn't been removed.
    X['Year^2'] = X['Year'] ** 2
    return X

process_and_fit_model(homeruns[['Year']], homeruns['Homeruns'], create_feature_columns_5a)

In [ ]:
...

In [ ]:
...

## Problem 6: Polynomial Regression

---

Run the cell below to generate the dataset for this problem. **Do not** modify the code in this cell!

In [ ]:
np.random.seed(23) # For reproducibility.

def sample_from_pop(n):
    x = np.linspace(-3, 3, n)
    y = (x**3) + (np.random.normal(0, 5, size=n))
    return pd.DataFrame({'x': x, 'y': y})

full = sample_from_pop(n=200)
full.plot(kind='scatter', x='x', y='y')

Here, we'll fit 25 polynomial models (one with degree 1, one with degree 2, ..., one with degree 25). But, as discussed in the PDF, we won't train each model on the entire dataset; instead, we'll split the data into training and test sets, train on the training set, and then evaluate on the test set.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets.
# The random_state ensures that we get the same "split" of the data each time we run this cell.
X_train, X_test, y_train, y_test = train_test_split(full[['x']], full['y'], test_size=0.2, random_state=15)

In [ ]:
X_train, y_train

In [ ]:
X_test, y_test

Instead of creating the features manually, like in Problem 5, we'll use some more advanced features in `sklearn` to create the features for us.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

Below, we create a **Pipeline** that first creates polynomial features of degree **20** (chosen as an example), and then fits a `LinearRegression` model on the transformed data.

In [ ]:
model = make_pipeline(PolynomialFeatures(degree=20, include_bias=False), LinearRegression())
model

In [ ]:
model.fit(X_train, y_train)

The model's training set and test set mean squared errors are below. (Unlike in Problem 5, we're using MSE instead of RMSE since the $y$-axis scale is already relatively small.)

In [ ]:
train_mse = mean_squared_error(y_train, model.predict(X_train))
test_mse = mean_squared_error(y_test, model.predict(X_test))
print(f"Training MSE: {train_mse:.2f}")
print(f"Test MSE: {test_mse:.2f}")

Note that the model performed significantly worse on the test set. Let's look at the model overlaid on both datasets.

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def plot_model_train_test(model, X_train, y_train, X_test, y_test):
    # Make prediction domain, but clip xs to [-2, 2.5]
    xs = np.linspace(-3, 3, 300).reshape(-1, 1)
    y_pred = model.predict(xs)
    
    # Set up figure with 1 row, 2 columns
    train_mse = mean_squared_error(y_train, model.predict(X_train))
    test_mse = mean_squared_error(y_test, model.predict(X_test))
    degree = model[0].degree
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Training Data (MSE for degree {degree}: {train_mse:.2f})", f"Test Data (MSE for degree {degree}: {test_mse:.2f})"))

    # (1) Left: training data
    fig.add_trace(
        go.Scatter(x=X_train.squeeze(), y=y_train, mode='markers', name="Train data"),
        row=1, col=1
    )
    # Model prediction curve (orange)
    fig.add_trace(
        go.Scatter(x=xs.squeeze(), y=y_pred, mode='lines', name="Predictions", line=dict(color='orange', width=4)),
        row=1, col=1
    )
    
    # (2) Right: test data
    fig.add_trace(
        go.Scatter(x=X_test.squeeze(), y=y_test, mode='markers', name="Test data"),
        row=1, col=2
    )
    # Model prediction curve (orange)
    fig.add_trace(
        go.Scatter(x=xs.squeeze(), y=y_pred, mode='lines', name="Predictions", showlegend=False, line=dict(color='orange', width=4)),
        row=1, col=2
    )
    
    # Clip x-axes to [-2, 2.5] in both subplots
    fig.update_xaxes(title_text="X", range=[-3, 3], row=1, col=1)
    fig.update_yaxes(title_text="y", row=1, col=1)
    fig.update_xaxes(title_text="X", range=[-3, 3], row=1, col=2)
    fig.update_yaxes(title_text="y", row=1, col=2)

    fig.update_layout(height=400, width=900, showlegend=True)
    fig.show(renderer='notebook')

plot_model_train_test(model, X_train, y_train, X_test, y_test)

Note that the model performed significantly worse on the test set. This, at some level, is expected; the model got to see the training data when choosing $\vec w^*$, and so it may have fit the training data too closely. But is 20 the best degree?

**Your Job**: Fill in the blanks below to complete the tasks mentioned in the PDF.

1. Fit 25 polynomial regression models on the training set (degree 1, degree 2, ..., degree 25).
1. For each model compute both its training and test mean squared error.
1. Create a line plot of the training and test mean squared error vs. degree. (There is helper code for this below.)
1. **Include screenshots of all of your code and the resulting plot in your Homework 8 PDF.**

That is, include screenshots of everything between the <span style="color:orange; font-weight:bold;">orange lines</span>.

<hr style="border: 0; height: 4px; background: orange;">

In [ ]:
def plot_two_lines(deg, train_mses, test_mses):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=deg, y=train_mses, mode='lines+markers', name='Train MSE'))
    fig.add_trace(go.Scatter(x=deg, y=test_mses, mode='lines+markers', name='Test MSE'))
    fig.update_layout(
        title="Training and Test MSE vs. Degree",
        xaxis_title='Degree',
        yaxis_title='MSE'
    )
    fig.show(renderer='notebook')

In [ ]:
...

<hr style="border: 0; height: 4px; background: orange;">

## Problem 7: One Hot Encoding in `sklearn`

---

In this problem, we'll aim to build a regression model that predicts the price of a house based on various features about it. The dataset we're using, originally compiled by Professor Dean De Cock at Truman State University **specifically for** teaching regression, contains information about houses sold in Ames, Iowa from 2006 to 2010.

Run the cell below to load in the data. A full data dictionary can be found [here](https://www.openintro.org/data/index.php?data=ames).

In [ ]:
houses = pd.read_csv('data/iowa.csv')
houses

Each row corresponds to a house. There are 80 columns (so 79 possible features); the last column contains the target variable, `SalePrice`.

In [ ]:
houses.columns

In [ ]:
fig = houses.plot(kind='scatter', x='Gr Liv Area', y='SalePrice')
fig.update_layout(title='Square Footage (Excluding Basement) vs. Sale Price')
fig.show(renderer='notebook')

In [ ]:
fig = houses['Neighborhood'].value_counts().plot(kind='bar')
fig.update_layout(title='Distribution of Neighborhoods')
fig.show(renderer='notebook')

As we learned in Problem 6, we should always perform a train/test split before building a model. Let's do that now.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(houses.drop(columns=['SalePrice']), houses['SalePrice'], test_size=0.2, random_state=15)

In Problem 6, we looked at how to create an `sklearn` Pipeline that first creates polynomial features using the `PolynomialFeatures` class, and then fits a `LinearRegression` model on the transformed data. But there, we wanted to create polynomial features out of every single "origin" feature, which was just the $x$-variable. But here, it might make sense to use `Gr Liv Area` (non-basement square footage) as a numerical feature as-is, and then one hot encode categorical features, like `Neighborhood`.

The big idea is that we need a way to tell `sklearn` what operations to apply on each feature. The solution is to use a `ColumnTransformer` object, which allows us to specify different operations for different columns. Let's look at an example of how this works.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

model = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(drop='first', handle_unknown='ignore'), ['Neighborhood']),
        (FunctionTransformer(lambda x: x, validate=False), ['Gr Liv Area']),
        remainder='drop'
    ),
    LinearRegression()
)

model

In [ ]:
model.fit(X_train, y_train)

You might have notice that after fitting the model, it appears blue (the same happened in the previous problem, too).

A lot of syntax went into defining `model`. Let's break it down:
1. First, `model` itself is a Pipeline. Its first step is a `ColumnTransformer`, which is the "controller" that tells `sklearn` what to do with each column. The second step is a `LinearRegression` model.
1. The `ColumnTransformer`, created using `make_column_transformer`, is instantiated using several tuples, each one containing the name of a transformation and a list of relevant columns.
    - We said apply `OneHotEncoder(drop='first', handle_unknown='ignore')` to the `Neighborhood` column.
    - `FunctionTransformer(lambda x: x, validate=False)` looks a little strange, but all it's saying is to **keep `Gr Liv Area` as is**.
    - `remainder='drop'` tells `sklearn` to drop the columns that are not mentioned in the list of tuples.
1. The `LinearRegression` model is the usual `LinearRegression` model.

We can peek into how the model was fit and transformed our data. First, we can look at its coefficients.

In [ ]:
# Access step 2 of the Pipeline, which is the LinearRegression model.
model[1]

In [ ]:
model[1].coef_

In [ ]:
# Total number of parameters in the model (including the intercept term, which is not accounted for above.)
len(model[1].coef_) + 1

Many of the coefficients above correspond to one hot encoded features. Let's look at the names of the one hot encoded features.

In [ ]:
model[0]

In [ ]:
model[0]['onehotencoder'].get_feature_names_out()

How many one hot encoded features did the model create?

In [ ]:
len(model[0]['onehotencoder'].get_feature_names_out())

But, in `X_train`, how many unique `Neighborhood` values are there?

In [ ]:
X_train['Neighborhood'].nunique()

Hmmm... what gives? The reason that our model produced one fewer one hot encoded features than there were unique `Neighborhood` values is that we told `sklearn` to drop one category of `Neighborhood` (the first one that it saw, to be exact). We did this to ensure that the resulting model's design matrix was of full column rank, meaning that all of its columns are linearly independent. As we've discussed in [Chapter 2.10](https://notes.eecs245.org/vectors-and-matrices/projection-2/) and, more recently, [Chapter 3.2](https://notes.eecs245.org/multiple-linear-regression/multiple-linear-regression/), this ensures that we have a unique solution for $\vec w^*$, our optimal parameter vector.

The cool thing is that to use our new model to make predictions, we can just use the usual `.predict` method, and don't need to worry about re-implementing the logic of one hot encoding.

For example, let's predict how much a house with 2550 square feet of living area in the `'CollgCr'` neighborhood is estimated to sell for.

In [ ]:
model.predict(pd.DataFrame({'Gr Liv Area': [2550], 'Neighborhood': ['CollgCr']}))

Just under \$285,000, it seems!


What if we make up a new neighborhood, like `'Ann Arbor'`, which definitely doesn't exist in the dataset?

In [ ]:
model.predict(pd.DataFrame({'Gr Liv Area': [2550], 'Neighborhood': ['Ann Arbor']}))

We get a `UserWarning`, but not an error, because we instantiated `OneHotEncoder` with `handle_unknown='ignore'`, which tells it to ignore any `Neighborhood`categories it sees in `.predict` that it didn't see in `.fit`.

What is this model's training and test mean squared error?

In [ ]:
train_mse = mean_squared_error(y_train, model.predict(X_train))
test_mse = mean_squared_error(y_test, model.predict(X_test))
print(f"Training MSE: {train_mse:.2f}")
print(f"Test MSE: {test_mse:.2f}")


**Your Job**: Below, build a Pipeline that:

- Creates `PolynomialFeatures` of degree 3 for the `Gr Liv Area` column (make sure to set `include_bias=False`, since `LinearRegression` will add its own intercept term).
- Uses the `Yr Sold` and `Year Built` columns as-is.
- One hot encodes **both** the `Neighborhood` column and the `MS SubClass` columns.
- Trains a `LinearRegression` model on the transformed data, ignoring/dropping any other columns.

In your notebook, include:

1. Screenshots of your code and the model's training and test mean squared errors.
2. The number of parameters in the model, including the intercept term; include screenshots of the code you used to find this number.
3. A 1-2 sentence explanation of why the model has this number of parameters.

<hr style="border: 0; height: 4px; background: orange;">

In [ ]:
...

<hr style="border: 0; height: 4px; background: orange;">

## Finish Line 🏁

Remember, this notebook does not need to be submitted to Gradescope! Make sure you've included the necessary screenshots for Problems 4-7 in your Homework 8 PDF.